In [5]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.docstore.document import Document
from unstructured.cleaners.core import remove_punctuation,clean,clean_extra_whitespace
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain



ModuleNotFoundError: No module named 'unstructured'

In [ ]:
def generate_document(url):
 "Given an URL, return a langchain Document to futher processing"
 loader = UnstructuredURLLoader(urls=[url],
 mode="elements",
 post_processors=[clean,remove_punctuation,clean_extra_whitespace])
 elements = loader.load()
 selected_elements = [e for e in elements if e.metadata['category']=="NarrativeText"]
 full_clean = " ".join([e.page_content for e in selected_elements])
 return Document(page_content=full_clean, metadata={"source":url})

In [ ]:
def summarize_document(url,llm):
 "Given an URL return the summary from OpenAI model"
 chain = load_summarize_chain(llm, chain_type="stuff")
 tmp_doc = generate_document(url)
 summary = chain.run([tmp_doc])
 return clean_extra_whitespace(summary)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain
from transformers import AutoTokenizer, pipeline, logging, AutoModelForCausalLM
import faiss

#Model
model_name_or_path = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,use_safetensors=True)

#Pipeline
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
summary=summarize_document(url,llm)